In [ ]:
from matplotlib import pyplot
from tf.keras.applications.nasnet.NASNetMobile import preprocess_input
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
import tensorflow as tf

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))




model = tf.keras.applications.NASNetMobile(
    input_shape=None,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
)

k = models.Sequential()
k.add(model)

for layer in k.layers:
	layer.trainable = False

k.add(Flatten())
k.add(BatchNormalization())
k.add(Dense(1024,activation='relu'))
k.add(Dropout(0.3))
k.add(Dense(512,activation='relu'))
k.add(Dropout(0.4))
k.add(Dense(3,activation='sigmoid'))
opt=SGD(lr=0.001, momentum=0.9)
k.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['acc',f1_m,precision_m, recall_m])
es = EarlyStopping(monitor='val_acc', mode='auto', verbose=1, patience=8)
mc = ModelCheckpoint('best_model_NasNetMobile_4L.h5', monitor='val_acc',mode='auto', verbose=1, save_best_only=True)

In [ ]:
def run_test_harness():
# define model

	# create data generators
	train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
	test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1./255)
	# prepare iterators

	train_it = train_datagen.flow_from_directory('DATA_PATH',
		class_mode='categorical', batch_size=64, target_size=(224, 224))
	test_it = test_datagen.flow_from_directory('DATA_PATH',
		class_mode='categorical', batch_size=64, target_size=(224, 224))
	# fit model
	history = k.fit_generator(train_it, steps_per_epoch=len(train_it),
		validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=1,callbacks=[es, mc])
	# evaluate model
	loss, accuracy, f1_score, precision, recall = k.evaluate_generator(test_it, steps=len(test_it), verbose=1)
	print('> %.3f' % (accuracy * 100.0))
	print('> %.3f' % (loss * 100.0))
	print('> %.3f' % (f1_score * 100.0))
	print('> %.3f' % (precision * 100.0))
	print('> %.3f' % (recall * 100.0))

	# plot diagnostic learning curves


	pyplot.subplot(211)
	pyplot.title('Loss')
	pyplot.plot(history.history['loss'], label='train')
	pyplot.plot(history.history['val_loss'], label='test')
	pyplot.tight_layout()
	pyplot.legend()
# plot accuracy during training
	pyplot.subplot(212)
	pyplot.plot(history.history['acc'], label='train')
	pyplot.plot(history.history['val_acc'], label='test')
	pyplot.title('Accuracy')
	pyplot.tight_layout()
	pyplot.legend()
	pyplot.show()

In [ ]:
run_test_harness()